In [ ]:
%run "../Part 1 - The Graph/Solution.ipynb"

In [ ]:
G = create_graph_of_people_nodes(people_df = people)

In [ ]:
def add_attribute_nodes(G:nx.DiGraph, attributes:list[str]) -> nx.DiGraph:
    """Add nodes to a people graph given the provided attributes
    
    Attribute nodes should be flagged differently than people nodes"""

    ...

    return G

def add_attribute_edges(G:nx.DiGraph) -> nx.DiGraph:
    """Add edges connecting each person to their attribute nodes"""

    ...

    return G

In [ ]:
def generate_graph_from(people:pd.DataFrame, with_attributes:list[str]=[]) -> nx.DiGraph: